In [182]:
# Jugador que se puede mover en cualquier lugar 
from dataclasses import dataclass, field
from typing import Dict, Union, List
import random as rd
from abc import ABC, abstractmethod

@dataclass
class Tiles(ABC):
    Name: str
    Position: List[int]
    def __post_init__(self):
        self.Position = self.Position

    @abstractmethod
    def Move(self):
        pass

    def CheckMovement(self, FuturePosition: int):
        if FuturePosition < 0 or FuturePosition > Grid.Size - 1:
            print('Invalid Movement')
            return False
        return True
    
@dataclass
class Wall(Tiles):
    
    def Move(self):
        pass
@dataclass
class Movement:
    Direction: str
    Grid: Grid = Grid()

    def nose():
        pass
    
@dataclass
class Player(Tiles):
    Name: str
    Position: List[int]
    def __post_init__(self):
        self.Position = self.Position
    
    def Move(self, Direction: str):
        if Direction.lower() == 'up' and self.CheckMovement(self.Position[0] - 1):
            self.Position[0] -= 1
        elif Direction.lower() == 'down' and self.CheckMovement(self.Position[0] + 1):
            self.Position[0] += 1
        elif Direction.lower() == 'left' and self.CheckMovement(self.Position[1] -1):
            self.Position[1] -= 1
        elif Direction.lower() == 'right' and self.CheckMovement(self.Position[1] + 1):
            self.Position[1] += 1
        else:
            print('Invalid Direction')
            return
        print(f'{self.Name} moved to {self.Position}')

    def CheckMovement(self, FuturePosition: int):
        if FuturePosition < 0 or FuturePosition > Grid.Size - 1:
            print('Invalid Movement')
            return False
        return True





@dataclass
class Grid:
    Size: int = 10
    Positions: List = field(default_factory=list)
    Walls: List = field(default_factory=list)



    def __post_init__(self):
        self.grid = [[' ' for _ in range(self.Size)] for _ in range(self.Size)] 

    def AddPlayer(self, Player: Player):
        self.Positions.append(Player)
    
    def Environment(self):
        for _ in range(10):    
            WallPosition = [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)]
            self.Walls.append(Wall('W', WallPosition))


    def CreateEnvironment(self):
        self.Environment()
        for Wall in self.Walls:
            self.grid[Wall.Position[0]][Wall.Position[1]] = Wall.Name
    
    def CheckColisions(self) -> bool:
        for Position in self.Positions:
            for Wall in self.Walls:
                if Position.Position == Wall.Position:
                    return False
                return True

    def UpdatePosition(self):
        if CheckColisions():
            self.grid = [[' ' for _ in range(self.Size)] for _ in range(self.Size)] 
            for Position in self.Positions:
                    self.grid[Position.Position[0]][Position.Position[1]] = Position.Name
            for Wall in self.Walls:
                self.grid[Wall.Position[0]][Wall.Position[1]] = Wall.Name               


    def UpdateGrid(self):
        self.UpdatePosition()
        self.ShowGrid()

    def ShowGrid(self):
        for row in self.grid:
            print(row)

    def ayuda(self):
        return self.Walls


In [177]:
Juego = Grid()
Juego.CreateEnvironment()
Jugador2 = Player('B', [2, 2])
Jugador1 = Player('A', [3, 4])

Juego.AddPlayer(Jugador1)
Juego.AddPlayer(Jugador2)


In [181]:
Jugador2.Move('right')
Juego.UpdateGrid()

B moved to [2, 6]
['W', 'W', ' ', ' ', 'W', ' ', ' ', 'W', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['W', ' ', ' ', ' ', ' ', ' ', 'B', ' ', 'W', ' ']
[' ', 'W', ' ', ' ', 'A', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', 'W', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']


In [166]:
Juego.ayuda()

[Wall(Name='W', Position=[2, 8]),
 Wall(Name='W', Position=[7, 5]),
 Wall(Name='W', Position=[6, 7]),
 Wall(Name='W', Position=[9, 6]),
 Wall(Name='W', Position=[1, 6]),
 Wall(Name='W', Position=[8, 6]),
 Wall(Name='W', Position=[5, 0]),
 Wall(Name='W', Position=[6, 8]),
 Wall(Name='W', Position=[1, 1]),
 Wall(Name='W', Position=[3, 4])]